In [1]:
from __future__ import print_function
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext


In [2]:
conf = SparkConf().setAppName("CS143 Project 2B")
conf = conf.setMaster("local[*]")
sc   = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
sc.addPyFile("cleantext.py")
#main(sqlContext)

comments = sqlContext.read.json("comments-minimal.json.bz2")
#comments.write.parquet("comments-minimal.json.bz2")
#submissions = sqlContext.read.json("submissions.json.bz2")
labels = sqlContext.read.csv("labeled_data.csv")

Py4JJavaError: An error occurred while calling o24.addFile.
: java.io.FileNotFoundException: File file:/C:/Users/Nabeel/Documents/Python%20Scripts/Project2/cleantext.py does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:611)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:824)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:421)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1528)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(Unknown Source)
	at java.lang.reflect.Method.invoke(Unknown Source)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngin

# Task 2

1. The functional dependencies on labeled_data.csv is input_id -> (labeldem, labelgop, labeldjt). The input_id is a superkey hence this is in BCNF. 

2. Comments has the schema (Author(char), Body(text), Can_Gild(boolean), Collapsed(Boolean), Contraversality(int), created_utc(int?), edited(boolean), gilded(int), id(char), is_submitter(boolean), link_id(char), parent_id(char), retrieved_on(int?), score(int), stickied(boolean), subreddit(text). We see there is some redundancy. For example, author determines Can_Gild. However, there don't seem to be many redundancies overall.


Write SQL query: SELECT comment, input_id, labeldel, labelgop, labeldjt 
                 FROM comment C INNER JOIN labeled_data L ON C.id = L.input_id



In [ ]:
labels.printSchema()
comments.printSchema()
#labels.show()

In [ ]:
comments.createOrReplaceTempView("Comments")
labels.createOrReplaceTempView("Labels")

X_y = sqlContext.sql("SELECT C.body, L._c0, L._c1, L._c2, L._c3 FROM Comments C INNER JOIN Labels L ON C.id = L._c0")

#X_y.show()

In [ ]:
from cleantext import sanitize

X_y.createOrReplaceTempView("Train")
just_comm = sqlContext.sql("SELECT body FROM Train")
#just_comm = just_comm.astype(str)


In [ ]:
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
#add the n_grams
from pyspark.sql.functions import udf
sanitize_udf = udf(sanitize)
sanitized_df = X_y.select("body", "_c1", "_c2", "_c3", sanitize_udf("body").alias("n_grams"))

concat_string_arrays_udf = udf(lambda out: (' '.join(out)).split(), ArrayType(StringType(), True))
concat_string_arrays_df = sanitized_df.select("body", "_c1", "_c2", "_c3", concat_string_arrays_udf("n_grams").alias("n_grams_formatted"))

In [ ]:
concat_string_arrays_df.limit(2).show(2, False)
#sanitized_df.show(2, False)

In [ ]:
#count vectorizer
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="n_grams_formatted", outputCol="features", minDF = 5, binary=True)

model = cv.fit(concat_string_arrays_df)

#result = model.transform(comments_df)
#result.show(truncate=False)

In [ ]:
result = model.transform(concat_string_arrays_df)
result.show(truncate=False)

In [ ]:
from pyspark.sql import functions as F
pos = result.select("body", "features", F.when(X_y._c1 == 1, 1).otherwise(0).alias("dem"), F.when(X_y._c2 == 1, 1).otherwise(0).alias("gop"), F.when(X_y._c3 == 1, 1).otherwise(0).alias("label"))
pos.show(20)
neg = result.select("body", "features", F.when(X_y._c1 == -1, 1).otherwise(0).alias("dem"), F.when(X_y._c2 == -1, 1).otherwise(0).alias("gop"), F.when(X_y._c3 == -1, 1).otherwise(0).alias("label"))
neg.show(20)

# Task 7

In [ ]:
# Bunch of imports (may need more)
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# Initialize two logistic regression models.
# Replace labelCol with the column containing the label, and featuresCol with the column containing the features.
poslr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10).setThreshold(.2)
neglr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10).setThreshold(.25)

In [ ]:
# This is a binary classifier so we need an evaluator that knows how to deal with binary classifiers.
posEvaluator = BinaryClassificationEvaluator()
negEvaluator = BinaryClassificationEvaluator()
# There are a few parameters associated with logistic regression. We do not know what they are a priori.
# We do a grid search to find the best parameters. We can replace [1.0] with a list of values to try.
# We will assume the parameter is 1.0. Grid search takes forever.
posParamGrid = ParamGridBuilder().addGrid(poslr.regParam, [1.0]).build()
negParamGrid = ParamGridBuilder().addGrid(neglr.regParam, [1.0]).build()

In [ ]:
# We initialize a 5 fold cross-validation pipeline.
posCrossval = CrossValidator(
    estimator=poslr,
    evaluator=posEvaluator,
    estimatorParamMaps=posParamGrid,
    numFolds=5)
negCrossval = CrossValidator(
    estimator=neglr,
    evaluator=negEvaluator,
    estimatorParamMaps=negParamGrid,
    numFolds=5)
# Although crossvalidation creates its own train/test sets for
# tuning, we still need a labeled test set, because it is not
# accessible from the crossvalidator (argh!)
# Split the data 50/50
posTrain, posTest = pos.randomSplit([0.5, 0.5])
negTrain, negTest = neg.randomSplit([0.5, 0.5])
# Train the models
print("Training positive classifier...")
posModel = posCrossval.fit(posTrain)
print("Training negative classifier...")
negModel = negCrossval.fit(negTrain)

# Once we train the models, we don't want to do it again. We can save the models and load them again later.
# posModel.save("www/pos.model")
# negModel.save("www/neg.model")

# Task 8

In [ ]:
# obtains data for testing
comments = sqlContext.read.json("comments-minimal.json.bz2")
comments.createOrReplaceTempView("Comments")
#comments.printSchema()
submissions = sqlContext.read.json("submissions.json.bz2")
submissions.createOrReplaceTempView("Submissions")
#submissions.printSchema()
#submissions.limit(1).show()

eval_data = sqlContext.sql("SELECT Comments.body as body, Comments.id as id FROM Comments JOIN Submissions ON Comments.link_id = Concat('t3_', Submissions.id)")



# Task 9

In [ ]:
# prepares data for testing

eval_data.createOrReplaceTempView("Eval_data")
#eval_data = sqlContext.sql("SELECT * FROM Eval_data")
eval_data = sqlContext.sql("SELECT body, id FROM Eval_data WHERE Eval_data.body NOT LIKE '/s' AND Eval_data.body NOT LIKE '&gt'")

#eval_data.where(not("/s" in eval_data.body) and not("&gt" in eval_data.body))

sanitize_udf = udf(sanitize)
#eval_data.show()
sanitized_df = eval_data.select("body", "id", sanitize_udf("body").alias("n_grams"))

concat_string_arrays_udf = udf(lambda out: (' '.join(out)).split(), ArrayType(StringType(), True))
concat_string_arrays_df = sanitized_df.select("body", "id", concat_string_arrays_udf("n_grams").alias("n_grams_formatted"))

#posTest = concat_string_arrays_df.select("body", "state", "created_utc", "score", "controversiality", "distinguished", "retrieved_on", "n_grams_formatted", F.when(concat_string_arrays_df._c3 == 1, 1).otherwise(0).alias("label"))
#negTest =  concat_string_arrays_df.select("body", "state", "created_utc", "score", "controversiality", "distinguished", "retrieved_on", "n_grams_formatted", F.when(concat_string_arrays_df._c3 == -1, 1).otherwise(0).alias("label"))


#run the model on the data




In [ ]:

posTestCounts = model.transform(concat_string_arrays_df)
posResult = posModel.transform(posTestCounts)
negTestCounts = model.transform(concat_string_arrays_df)
negResult = negModel.transform(negTestCounts)

In [ ]:

posResult.show()
#posResult.write.format("posResult.csv")
negResult.show()
#negResult.write.format("negResult.csv")

In [ ]:
posResult.show()

# Task 10

In [ ]:
posResult.createOrReplaceTempView("PosResult")
negResult.createOrReplaceTempView("NegResult")
resultCount = sqlContext.sql("SELECT COUNT(*) FROM PosResult")
resultCount.show()
posResultCount = sqlContext.sql("SELECT COUNT(*) FROM PosResult WHERE prediction == 1")
posResultCount.show()
negResultCount = sqlContext.sql("SELECT COUNT(*) FROM NegResult WHERE prediction == 1")
negResultCount.show()

In [ ]:
comments.createOrReplaceTempView("Comments")
posResultWComment = sqlContext.sql("SELECT Comments.body as body, Comments.id as id, Comments.author_flair_text as state, Comments.created_utc as created_utc, Comments.score as score, Comments.controversiality as controversiality, Comments.distinguished as distinguished, PosResult.prediction as prediction FROM Comments JOIN PosResult ON Comments.id == PosResult.id")
posResultWComment.createOrReplaceTempView("PosResultWComment")
negResultWComment = sqlContext.sql("SELECT Comments.body as body, Comments.id as id, Comments.author_flair_text as state, Comments.created_utc as created_utc, Comments.score as score, Comments.controversiality as controversiality, Comments.distinguished as distinguished, NegResult.prediction as prediction FROM Comments JOIN NegResult ON Comments.id == PosResult.id")
negResultWComment.createOrReplaceTempView("NegResultWComment")
#percent by day
posDays = sqlContext.sql("SELECT created_utc, SUM(prediction) / COUNT(prediction) as pct FROM PosResultWComment GROUP BY created_utc")
posDays.show()
negDays = sqlContext.sql("SELECT created_utc, SUM(prediction) / COUNT(prediction) as pct FROM NegResultWComment GROUP BY created_utc")
negDays.show()

In [ ]:
posState = sqlContext.sql("SELECT state, SUM(prediction) / COUNT(prediction) as pct FROM PosResultWComment GROUP BY state")
posState.show()
negState = sqlContext.sql("SELECT state, SUM(prediction) / COUNT(prediction) as pct FROM NegResultWComment GROUP BY state")
negState.show()

In [ ]:
posScores = sqlContext.sql("SELECT id, SUM(prediction) / COUNT(prediction) as pct FROM PosResultWComment GROUP BY id")
posScores.show()
negScores = sqlContext.sql("SELECT id, SUM(prediction) / COUNT(prediction) as pct FROM NegResultWComment GROUP BY id")
negScores.show()

In [ ]:
posScores = sqlContext.sql("SELECT score, SUM(prediction) / COUNT(prediction) as pct FROM PosResultWComment GROUP BY score")
posScores.show()
negScores = sqlContext.sql("SELECT score, SUM(prediction) / COUNT(prediction) as pct FROM NegResultWComment GROUP BY score")
negScores.show()

In [ ]:
posControversiality = sqlContext.sql("SELECT controversiality, SUM(prediction) / COUNT(prediction) as pct FROM PosResultWComment GROUP BY controversiality")
posControversiality.show()
negControversiality = sqlContext.sql("SELECT controversiality, SUM(prediction) / COUNT(prediction) as pct FROM NegResultWComment GROUP BY controversiality")
negControversiality.show()

In [ ]:
posDistinguished = sqlContext.sql("SELECT distinguished, SUM(prediction) / COUNT(prediction) as pct FROM PosResultWComment GROUP BY distinguished")
posDistinguished.show()
negDistinguished = sqlContext.sql("SELECT distinguished, SUM(prediction) / COUNT(prediction) as pct FROM NegResultWComment GROUP BY distinguished")
negDistinguished.show()